In [ ]:
import traceback
%load_ext autoreload
%autoreload 2

from ipywidgets import FloatProgress, Layout
from IPython.display import display
import micasense.imageset as imageset
import micasense.capture as capture
import os, sys, re, datetime
from pathlib import Path
import exiftool
import micasense.imageutils as imageutils
from osgeo import gdal
import numpy as np
from numpy import float32
from skimage.transform import ProjectiveTransform
import pandas as pd
from mapboxgl.viz import CircleViz
from mapboxgl.utils import df_to_geojson, create_color_stops

# --- Initial Setup ---
exiftool_dir = r"C:\exiftool\exiftool.exe"
if os.name == 'nt':
    os.environ['PATH'] = exiftool_dir + os.pathsep + os.environ['PATH']

panSharpen = True
write_exif_to_individual_stacks = False
useDLS = True
overwrite = False
generateThumbnails = True

# --- Path Definitions ---
# 📌 SET YOUR PARENT AND PANEL PATHS HERE
# Define the parent path containing all the subdirectories (e.g., 000, 001, 002...)
parent_path = Path(r"G:\Supherb Drone Images\9-20-25 88-21 Merced Field\0001SET")

# Define the path for panel calibration images
imagePathPanel = Path(r"G:\Supherb Drone Images\9-20-25 88-21 Merced Field\0000SET\000")

# --- Panel & Camera Calibration ---
panelNames = list(imagePathPanel.glob('IMG_0003_*.tif'))
panelNames = [x.as_posix() for x in panelNames] if panelNames else None

if panelNames:
    panelCap = capture.Capture.from_filelist(panelNames)
else:
    panelCap = None

if panelCap is None:
    raise ValueError("Panel capture could not be loaded. Please check the panel image path.")

cam_model = panelCap.camera_model
panchroCam = cam_model in ['RedEdge-P', 'Altum-PT']
if not panchroCam:
    panSharpen = False

if len(panelCap.camera_serials) > 1:
    cam_serial = "_".join(panelCap.camera_serials)
else:
    cam_serial = panelCap.camera_serial
print(f"Camera model: {cam_model}, Serial number: {cam_serial}")

if panelCap.panel_albedo() is not None and not any(v is None for v in panelCap.panel_albedo()):
    panel_reflectance_by_band = panelCap.panel_albedo()
else:
    panel_reflectance_by_band = [0.49] * len(panelCap.eo_band_names())

panel_irradiance = panelCap.panel_irradiance(panel_reflectance_by_band)
img_type = "reflectance" if useDLS else "radiance"

# --- Load Alignment Warp Matrices ---
if panchroCam:
    warp_matrices_filename = f"{cam_serial}_warp_matrices_SIFT.npy"
else:
    warp_matrices_filename = f"{cam_serial}_warp_matrices_opencv.npy"

if Path(warp_matrices_filename).is_file():
    print(f"Found existing warp matrices: {warp_matrices_filename}")
    load_warp_matrices = np.load(warp_matrices_filename, allow_pickle=True)
    if panchroCam:
        warp_matrices = [ProjectiveTransform(matrix=matrix.astype('float64')) for matrix in load_warp_matrices]
    else:
        warp_matrices = [matrix.astype('float32') for matrix in load_warp_matrices]
    print("Successfully loaded warp matrices.")
else:
    warp_matrices = None
    print(f"WARNING: No warp matrices found at '{warp_matrices_filename}'. Alignment may be suboptimal.")


# --- Get list of subdirectories to process ---
subdirectories_to_process = sorted([d for d in parent_path.iterdir() if d.is_dir()])
print(f"\nFound {len(subdirectories_to_process)} subdirectories to process in '{parent_path.name}'.")

try:
    # --- Generate Map Visualization for the FIRST Subdirectory as a Preview ---
    if subdirectories_to_process:
        print("\nGenerating map preview from first directory...")
        
        first_dir = subdirectories_to_process[0]
        
        # Check if the directory is empty first
        if not any(first_dir.glob('*.tif')):
            print(f"Skipping map preview: First directory '{first_dir.name}' contains no .tif files.")
        else:
            preview_imgset = imageset.ImageSet.from_directory(first_dir)
            data, columns = preview_imgset.as_nested_lists()
            df = pd.DataFrame.from_records(data, index='timestamp', columns=columns)
            
            token = 'pk.eyJ1Ijoic2VudGlsbGlhbmFseXRpY3MiLCJhIjoiY204aTJsa3Z4MDk1YTJxb241dzdpeXd2ZSJ9.J-EmczpBD5bvvI8NiVG8kQ'
            geojson_data = df_to_geojson(df, columns[3:], lat='latitude', lon='longitude')
            
            # This corrected block prevents the ValueError
            min_val = df['dls-yaw'].min()
            max_val = df['dls-yaw'].max()
    
            if min_val == max_val:
                # If all values are the same, just use one color
                print("Warning: All 'dls-yaw' values are identical. Map will show a single color.")
                color_stops = [[min_val, 'blue']]
            else:
                # Create 5 evenly spaced breaks for a smooth gradient
                breaks = np.linspace(min_val, max_val, 5).tolist()
                color_stops = create_color_stops(breaks, colors='YlOrRd')
    
            viz = CircleViz(geojson_data, access_token=token, color_property='dls-yaw',
                            color_stops=color_stops,
                            center=[df['longitude'].median(), df['latitude'].median()],
                            zoom=16, height='600px',
                            style='mapbox://styles/mapbox/satellite-streets-v9')
            viz.show()
            del preview_imgset, data, columns, df # Clean up memory

except Exception as e:
    print(f"\n{e}")

# --- Main Processing Loop ---
print("\n--- Starting Batch Processing ---")

for imagePath in subdirectories_to_process:
    try: 
    
    
        print(f"\n{'='*50}\nProcessing directory: {imagePath}\n{'='*50}")
    
        # Check if directory contains TIFF files to avoid errors
        if not list(imagePath.glob('*.tif')):
            print(f"Skipping '{imagePath.name}' - No .tif files found.")
            continue
    
        # Set output paths relative to the current subdirectory
        outputPath = imagePath / 'stacks'
        thumbnailPath = outputPath / 'thumbnails'
    
        # Create output directories if they don't exist
        if not outputPath.exists():
            outputPath.mkdir(parents=True)
        if generateThumbnails and not thumbnailPath.exists():
            thumbnailPath.mkdir(parents=True)
            
        f_load = FloatProgress(min=0, max=1, layout=Layout(width='100%'), description="Loading")
        display(f_load)
        def update_f_load(val):
            if (val - f_load.value) > 0.005 or val == 1:
                f_load.value=val
        
        # Create ImageSet from the current subdirectory
        imgset = imageset.ImageSet.from_directory(imagePath, progress_callback=update_f_load)
        update_f_load(1.0)
        print(f"Loaded {len(imgset.captures)} captures from '{imagePath.name}'.")
    
        # This list will hold EXIF data for the current directory's images
        exif_list = []
        
        f_save = FloatProgress(min=0, max=1, layout=Layout(width='100%'), description="Saving")
        display(f_save)
        def update_f_save(val):
            f_save.value=val
    
        # Start processing captures for the current directory
        start_time = datetime.datetime.now()
        for i, capture in enumerate(imgset.captures):
            try:
                # Dynamically get path parts from the current imagePath
                path_parts = str(imagePath.resolve()).split(os.sep)
                
                # This assumes a consistent folder structure. Adjust indices if needed.
                # e.g., .../field_name/trial_name/parent_folder/set_number
                field_name = path_parts[-4] if len(path_parts) >= 4 else 'field'
                trial_name = path_parts[-3] if len(path_parts) >= 3 else 'trial'
                set_number = path_parts[-1] # The last part of the path is the set number
                
                # Clean folder names for use in filenames
                clean_field_name = re.sub(r'\W+', '_', field_name)
                clean_trial_name = re.sub(r'\W+', '_', trial_name)
                clean_set_number = re.sub(r'\W+', '_', set_number)
        
                # Construct unique output filenames
                outputFilename = f"{clean_field_name}_{clean_trial_name}_{clean_set_number}_{str(i).zfill(4)}_{capture.uuid}.tif"
                thumbnailFilename = f"{clean_field_name}_{clean_trial_name}_{clean_set_number}_{str(i).zfill(4)}_{capture.uuid}.jpg"
                fullOutputPath = outputPath / outputFilename
                fullThumbnailPath = thumbnailPath / thumbnailFilename
        
                if not fullOutputPath.exists() or overwrite:
                    if len(capture.images) == len(imgset.captures[0].images):
                        if panchroCam:
                            # For Altum-PT / RedEdge-P
                            capture.radiometric_pan_sharpened_aligned_capture(
                                warp_matrices=warp_matrices,
                                irradiance_list=capture.dls_irradiance(),
                                img_type=img_type
                            )
                        else:
                            # For other cameras
                            irradiance = panel_irradiance + [0] if panel_irradiance else None
                            capture.create_aligned_capture(irradiance_list=irradiance, warp_matrices=warp_matrices)
        
                        exif_list.append(imageutils.prepare_exif_for_stacks(capture, str(fullOutputPath)))
                        capture.save_capture_as_stack(str(fullOutputPath), pansharpen=panSharpen, sort_by_wavelength=True, write_exif=write_exif_to_individual_stacks)
                        
                        if generateThumbnails:
                            capture.save_capture_as_rgb(str(fullThumbnailPath))
                
                capture.clear_image_data()
                update_f_save(float(i+1) / len(imgset.captures))
            except Exception as e:
                print(e)
                continue
        
        end_time = datetime.datetime.now()
        proc_time = end_time - start_time
        print(f"Alignment and stacking time: {proc_time}")
        if proc_time.total_seconds() > 0:
            rate = len(imgset.captures) / proc_time.total_seconds()
            print(f"Processing rate: {rate:.2f} captures/sec")
    
        if not write_exif_to_individual_stacks and exif_list:
            print("Writing EXIF metadata to stacked files...")
            start_exif_time = datetime.datetime.now()
            for exif in exif_list:
                imageutils.write_exif_to_stack(existing_exif_list=exif)
            end_exif_time = datetime.datetime.now()
            print(f"EXIF writing time: {end_exif_time - start_exif_time}")
    except Exception as e:
        print(e)
        continue
        
print("\n--- All processing complete. ---")

Camera model: RedEdge-P, Serial number: PR03-2310192-MS
Found existing warp matrices: PR03-2310192-MS_warp_matrices_SIFT.npy
Successfully loaded warp matrices.

Found 3 subdirectories to process in '0001SET'.

Generating map preview from first directory...


C:\Users\jesus\anaconda3\envs\micasense\lib\site-packages\IPython\core\display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")



--- Starting Batch Processing ---

Processing directory: G:\Supherb Drone Images\9-20-25 88-21 Merced Field\0001SET\000


FloatProgress(value=0.0, description='Loading', layout=Layout(width='100%'), max=1.0)

Loaded 200 captures from '000'.


FloatProgress(value=0.0, description='Saving', layout=Layout(width='100%'), max=1.0)